FONCTIONS

In [1]:
import pandas as pd
import numpy as np
import ast
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
# modèle
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def bool_colonne(df, df_recherche2, colonne):

    df[colonne] = df[colonne].astype(str)
    df[colonne] = df[colonne].apply(lambda x : x.replace('[', '').replace(']', '').replace("'", '').replace(", ", ","))
    df[colonne] = df[colonne].apply(lambda x : x.split(','))

    df_recherche2[colonne] = df_recherche2[colonne].astype(str)
    df_recherche2[colonne] = df_recherche2[colonne].apply(lambda x : x.replace('[', '').replace(']', '').replace("'", '').replace(", ", ","))
    df_recherche2[colonne] = df_recherche2[colonne].apply(lambda x : x.split(','))

    tous_les_genres = []

    for element in df_recherche2[colonne].iloc[0] :
        if colonne == 'acteur_out_KNN':
            if element in liste_acteurs_recurrents:
                tous_les_genres.append(element)
                tous_les_genres = list(set(tous_les_genres))
        elif colonne == 'realisateurs_out_KNN':
            if element in liste_realisateurs_recurrents:
                tous_les_genres.append(element)
                tous_les_genres = list(set(tous_les_genres))
        elif colonne == 'production_companies_name_out_KNN':
            if element in liste_production_recurrents:
                tous_les_genres.append(element)
        else:
            tous_les_genres.append(element)  

    for element2 in tous_les_genres:
        df[f'{colonne[0:-8]}_{element2}'] = df[f'{colonne}'].apply(lambda x: element2 in x)

    return df, tous_les_genres

In [3]:
# FONCTION 1

def encodage_X(X, type, poids):
  from sklearn.preprocessing import StandardScaler
  index = X.index
  X_num = X.select_dtypes('number')

  if type == 'standard':
      from sklearn.preprocessing import StandardScaler
      SN = StandardScaler()
      X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns)

  else:
      from sklearn.preprocessing import MinMaxScaler
      SN = MinMaxScaler()
      X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns)

  X_num_SN = X_num_SN.mul(poids, axis = 1)
  X_encoded = X_num_SN

  X_encoded = X_encoded.dropna()

  return X_encoded, SN

# FONCTION 2

def evaluate_k(X_encoded, k_range):
    """
    Évalue différentes valeurs de k en utilisant la somme des distances aux voisins
    et le score de silhouette comme métriques.

    Args:
        X_encoded (DataFrame): Données normalisées
        k_range (range): Plage de valeurs de k à tester

    Returns:
        tuple: (distances moyennes, scores de silhouette)
    """
    from sklearn.metrics import silhouette_score
    from sklearn.cluster import KMeans

    avg_distances = []
    silhouette_scores = []

    for k in k_range:
        # Calcul des distances moyennes pour chaque k
        from sklearn.neighbors import NearestNeighbors
        model = NearestNeighbors(n_neighbors=k)
        model.fit(X_encoded)
        distances, _ = model.kneighbors(X_encoded)
        avg_distances.append(np.mean(distances))

        # Calcul du score de silhouette
        # Nous utilisons KMeans pour créer des clusters et évaluer la qualité
        kmeans = KMeans(n_clusters=k, random_state=42)
        clusters = kmeans.fit_predict(X_encoded)
        if k > 1:  # Le score de silhouette nécessite au moins 2 clusters
            silhouette_scores.append(silhouette_score(X_encoded, clusters))
        else:
            silhouette_scores.append(0)

    return avg_distances, silhouette_scores

# FONCTION 3

def encodage_predict(df_a_predire, SN, poids, X_encoded):
  X_num = df_a_predire.select_dtypes('number')

  X_num_SN = pd.DataFrame(SN.transform(X_num), columns=X_num.columns).reset_index(drop=True)
  X_num_SN = X_num_SN.mul(poids, axis = 1)
  
  X_encoded_predire = X_num_SN

  df_predict = X_encoded_predire

  # DataFrame vide qui a les mêmes colonnes que X_encoded
  df_final = pd.DataFrame(columns=X_encoded.columns)

  # On veut que le DataFrame ait le même nombre de lignes que df_predict
  df_final = df_final.reindex(index=df_predict.index)
  # On met tous les NaN à False
  df_final = df_final.fillna(False)

  # On parcourt chaque colonne de df_predict
  # Si la colonne est présente dans X_encoded alors on la garde
  # Sinon, on la met à False
  for column in df_predict.columns:
    if column in X_encoded.columns:
      df_final[column] = df_predict[column]

  return df_final

# FONCTION 4

def pokemons_similaires(X, film_id, model, SN, poids, X_encoded, df):

  # Vérifier si le Pokémon existe dans le dataset
  if film_id not in X['film_id_out_KNN'].values:
      return f"Le Pokémon {film_id} n'est pas dans le dataset."

  # Récupérer les caractéristiques du Pokémon
  pokemon = X[X['film_id_out_KNN'] == film_id]

  # Je recopie ce qu'on a fait avant:
  caract_pokemon = X[X['film_id_out_KNN'] == film_id]

  caract_pokemon_encoded = encodage_predict(caract_pokemon, SN, poids, X_encoded)

  distances, indices = model.kneighbors(caract_pokemon_encoded)

  return df.iloc[indices[0]].reset_index(drop=True)

IMPORTS

In [4]:
# df = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip', na_values = ['\\N'])
df = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip')
df = df.fillna(0)


/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_25699/3122574319.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip')


In [5]:
df_acteurs = df.copy()
df_realisateurs = df.copy()
df_production = df.copy()
df_acteurs_recurrents = df.copy()
df_realisateurs_recurrents = df.copy()
df_production_reccurent = df.copy()
df_recherche = df.copy()

ACTEURS, REALISATEURS ET PRODUCTION RECURRENTS

In [6]:
liste_acteurs_recurrents = []
for element in df.columns:
    if 'acteur' in element:
        liste_acteurs_recurrents.append(element[7:])
        
liste_realisateurs_recurrents = []
for element in df.columns:
    if 'realisateur' in element:
        liste_realisateurs_recurrents.append(element[13:])

liste_production_companies_name_recurrents = []
for element in df.columns:
    if 'production_companies_name' in element:
        liste_production_companies_name_recurrents.append(element[26:])

In [7]:
# CHOIX DES CARACTERISTIQUES

caracteristiques = []

for element in df.columns:
    if 'out_KNN' not in element:
        caracteristiques.append(element)

caracteristiques_num = []

for element in df.select_dtypes(include = 'number').columns:
    if 'out_KNN' not in element:
        caracteristiques_num.append(element)

In [8]:
# METTRE UNIQUEMENT POUR LES COLONNES NUMERIQUES

poids_list = pd.DataFrame(columns = caracteristiques_num, index = ['poids'])

colonne_cle = 10
tres_important = 8
important = 4
bof = 2
rien = 1

poids = {
 'popularity' : colonne_cle,
 'year_exact' : important,
 'Decennie' : colonne_cle,
 'runtime_exact' : rien,
 'vote_exact' : important,
 'arrondi_vote_exact' : colonne_cle,
# 'vote_count_mean' : important,
 'prod_US' : important,
 'prod_FR' : important
}

for element in df.select_dtypes(include = 'number').columns:
    if "production_companies_name" in element:
        poids.update({element : important})
    elif "acteur_{" in element:
        poids.update({element : tres_important})
    elif "realisateurs_" in element:
        poids.update({element : tres_important})
    elif "genre_" in element:
        poids.update({element : colonne_cle})
    # else:
    #      poids.update({element : rien})



for element in poids_list.columns:
    if element not in poids.keys():
        poids.update({element : rien})

In [9]:
# RECHERCHE DU TITRE

df_recherche = df.copy()
df_recherche['title_out_KNN'] = df_recherche['title_out_KNN'].apply(lambda x : x.lower())
recherche = 'avatar 2'
recherche2 = recherche.lower().split(" ")

for element in recherche2:
    df_recherche2 = df_recherche[df_recherche['title_out_KNN'].str.contains(element)]
    df_recherche = df_recherche2

film_id = df_recherche2['film_id_out_KNN'].iloc[0]

In [10]:
# # RECHERCHE DU REALISATEURS

# df_realisateurs_recherche = pd.DataFrame(df_realisateurs[df_realisateurs['film_id_out_KNN'].str.contains(film_id) == True]['primaryName']).drop_duplicates()
# df_realisateurs_recherche

In [11]:
# BASES

X = df[caracteristiques]

df_a_predire = df[df['film_id_out_KNN'] == film_id]
search = df_a_predire['title_out_KNN'].iloc[0]
# df_a_predire = df_a_predire.drop('title_len_out_KNN', axis = 1)
df_a_predire = df_a_predire[caracteristiques]


k_range = (3,5)

In [12]:
# FONCTION 1

X_encoded, SN = encodage_X(X, 'standard', poids)

# # FONCTION 2

avg_distances, silhouette_scores = evaluate_k(X_encoded, k_range)

# # FONCTION 3

df_final = encodage_predict(df_a_predire, SN, poids, X_encoded)

# # FONCTION 4

# On choisit k
k=8

model = NearestNeighbors(n_neighbors=k, metric='euclidean')
model.fit(X_encoded)

In [76]:
caracteristiques.append('film_id_out_KNN')
resultat = pokemons_similaires(df[caracteristiques], film_id, model, SN, poids, X_encoded, df)
choix = pd.DataFrame(df[df['title_out_KNN'] == search])

# # choix2 = choix.drop(columns = choix.columns[22:])
# # resultat2 = resultat.drop(columns = resultat.columns[22:])

final = pd.concat([choix, resultat])
final = final.drop(0)

caracteristiques.remove('film_id_out_KNN')

final.transpose()
final['title_out_KNN']

ValueError: Input X contains NaN.
NearestNeighbors does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

ACTEURS ET REALISATEURS DU FILM

In [31]:
liste_acteurs = []
for element in df_a_predire.columns:
    if 'acteur' in element and df_a_predire[element].iloc[0] == 1:
        liste_acteurs.append(element[7:])
        
liste_realisateurs = []
for element in df_a_predire.columns:
    if 'realisateur' in element and df_a_predire[element].iloc[0] == 1:
        liste_realisateurs.append(element[13:])

liste_production_companies_name = []
for element in df_a_predire.columns:
    if 'production_companies_name' in element and df_a_predire[element].iloc[0] == 1:
        liste_production_companies_name.append(element[26:])

In [ ]:
df_acteurs = df_acteurs[~(df_acteurs['film_id_out_KNN'] == film_id)]
df_acteurs['acteur_out_KNN'] = df_acteurs['acteur_out_KNN'].astype(str)
df_acteurs['acteur_out_KNN'] = df_acteurs['acteur_out_KNN'].apply(lambda x : str(x))
df_acteurs = df_acteurs.sort_values(by = 'vote_exact', ascending = False)

dico_acteur = {}

for element in liste_acteurs:
    dico_acteur.update({df_acteurs[df_acteurs['acteur_out_KNN'].str.contains(element.lower())].sort_values(by = 'vote_exact', ascending = False).head()['vote_exact'].mean() : element})

sorted_dico_acteur = dict(sorted(dico_acteur.items(), reverse = True))

for element in sorted_dico_acteur.values():
    element = element.lower()
    display(df_acteurs[df_acteurs['acteur_out_KNN'].str.contains(element)].head())

In [ ]:
df_realisateurs = df_realisateurs[~(df_realisateurs['film_id_out_KNN'] == film_id)]
df_realisateurs['realisateurs_out_KNN'] = df_realisateurs['realisateurs_out_KNN'].astype(str)
df_realisateurs['realisateurs_out_KNN'] = df_realisateurs['realisateurs_out_KNN'].apply(lambda x : str(x))
df_realisateurs = df_realisateurs.sort_values(by = 'vote_exact', ascending = False)

dico_realisateur = {}

for element in liste_realisateurs:
    dico_realisateur.update({df_realisateurs[df_realisateurs['realisateurs_out_KNN'].str.contains(element.lower())].sort_values(by = 'vote_exact', ascending = False).head()['vote_exact'].mean() : element})

sorted_dico_realisateur = dict(sorted(dico_realisateur.items(), reverse = True))

for element in sorted_dico_realisateur.values():
    element = element.lower()
    display(df_realisateurs[df_realisateurs['realisateurs_out_KNN'].str.contains(element)].head())

In [64]:
df_production = df_production[~(df_production['film_id_out_KNN'] == film_id)]
df_production['production_companies_name_out_KNN'] = df_production['production_companies_name_out_KNN'].astype(str)
df_production['production_companies_name_out_KNN'] = df_production['production_companies_name_out_KNN'].apply(lambda x : str(x))
df_production = df_production.sort_values(by = 'vote_exact', ascending = False)

dico_production = {}

for element in liste_production_companies_name:
    dico_production.update({df_production[df_production['production_companies_name_out_KNN'].str.contains(element.lower())].sort_values(by = 'vote_exact', ascending = False).head()['vote_exact'].mean() : element})

sorted_dico_production = dict(sorted(dico_production.items(), reverse = True))

for element in sorted_dico_production.values():
    element = element.lower()
    display(df_production[df_production['production_companies_name_out_KNN'].str.contains(element)].head())

,production_companies_name_out_KNN,genre_out_KNN,title_out_KNN,film_id_out_KNN,arrondi_vote_exact,popularity,year_exact,Decennie,runtime_exact,vote_exact,...,production_companies_name_bernsbrothersproductions,production_companies_name_edfriendlyproductions,production_companies_name_sktelecom,production_companies_name_gruppobema,production_companies_name_lefooleinc.,production_companies_name_zuludawnnv,production_companies_name_filmel,production_companies_name_l.a.reidmedia,production_companies_name_apertedevue,production_companies_name_filmgalerie451
6532,['marvelstudios'],"['sciencefiction', 'adventure', 'drama', 'acti...",Avengers:Phasefinale,tt4154796,8.0,101.337,2019.0,2010.0,181.0,8.35,...,0,0,0,0,0,0,0,0,0,0
7279,['marvelstudios'],"['sci-fi', 'sciencefiction', 'adventure', 'act...",Avengers:laguerredelinfini,tt4154756,8.0,205.036,2018.0,2010.0,149.0,8.35,...,0,0,0,0,0,0,0,0,0,0
6558,"['marvelstudios', 'kevinfeigeproductions']","['sciencefiction', 'adventure', 'comedy', 'act...",LesgardiensdelagalaxieVol.3,tt6791350,8.0,2520.308,2023.0,2020.0,150.0,8.10,...,0,0,0,0,0,0,0,0,0,0
6429,"['marvelstudios', 'pascalpictures', 'columbiap...","['sciencefiction', 'fantasy', 'adventure', 'ac...",Spider-Man:SansRetour,tt10872600,8.0,262.366,2021.0,2020.0,148.0,8.10,...,0,0,0,0,0,0,0,0,0,0
6566,['marvelstudios'],"['sciencefiction', 'adventure', 'comedy', 'act...",Lesgardiensdelagalaxie,tt2015381,8.0,255.418,2014.0,2010.0,121.0,7.95,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Définition de la plage de k à tester

k_range = range(1,30)

# Évaluation des différentes valeurs de k
avg_distances, silhouette_scores = evaluate_k(X_encoded, k_range)

# Création d'une visualisation pour aider à choisir k
plt.figure(figsize=(12, 5))

# Premier graphique : Distance moyenne aux voisins
plt.subplot(1, 2, 1)
plt.plot(k_range, avg_distances, 'bo-')
plt.xlabel('Nombre de voisins (k)')
plt.ylabel('Distance moyenne aux voisins')
plt.title('Distance moyenne en fonction de k')
plt.grid(True)

# Second graphique : Score de silhouette
plt.subplot(1, 2, 2)
plt.plot(k_range[1:], silhouette_scores[1:], 'ro-')  # On commence à k=2
plt.xlabel('Nombre de voisins (k)')
plt.ylabel('Score de silhouette')
plt.title('Score de silhouette en fonction de k')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
#EXPORTER MODEL

import pickle

# Sauvegarder le modèle
with open('mon_modele.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# IMPORTER MODEL

import pickle

# Charger le modèle
with open('mon_modele.pkl', 'rb') as f: #là vous mettez l'emplacement et le nom de votre fichier pkl
    model_charge = pickle.load(f)

In [211]:
# X_train, X_test, y_train, y_test = train_test_split(df_films.drop('title_out_KNN', axis=1), df_films['title_out_KNN'], test_size=0.75)

# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

# param_grid = {"n_neighbors": range(20)}

# # Perform grid search with cross-validation
# gscv = GridSearchCV(model, param_grid, cv=5)
# gscv.fit(X_train, y_train)


# print("Best params:", gscv.best_params_)
# print("Best cross-validation score:", gscv.best_score_)

In [ ]:
# df_films = df.copy()

# df_films['production_companies_name2'] = df_films['production_companies_name'].apply(lambda x: x.replace("[", '').replace("]", '').replace("'", ''))

# df_films['production_companies_name3'] = df_films['production_companies_name2'].apply(lambda x : x.split(','))

# tous_les_genres = set()
# tous_les_genres2 = set()

# for element in df_films['production_companies_name3']:
#   tous_les_genres.update(element)

# tous_les_genres_list = list(tous_les_genres)

# tous_les_genres_list.remove("")

# for n in range(len(tous_les_genres_list)):
#   tous_les_genres_list[n] = tous_les_genres_list[n].replace(" ","")

# tous_les_genres2_list = list(tous_les_genres2)
# tous_les_genres2_list.remove("")

# df_films['production_companies_name4'] = df_films['production_companies_name'].apply(lambda x: x.replace(" ", ''))

# new_dict = {}

# for element in tous_les_genres_list:
#     if len(df_films[df_films['production_companies_name2'].str.contains(element) == True]) >= 500:
#       new_dict.update({element : len(df_films[df_films['production_companies_name2'].str.contains(element) == True])})

# new_dict_sorted = dict(sorted(new_dict.items(), key=lambda x:x[1]))

# for genre in new_dict_sorted.keys():
#   df_films[f'production_companies_name_{genre}'] = df_films['production_companies_name4'].apply(lambda x: genre in x)

In [ ]:
# df_LN = df[df['vote_count_mean'] != 0].drop(['adult', 'production_countries', 'status', 'tagline', 'production_companies_name', 'production_companies_country', 'titleType', 'region', 'Decennie', 'genre', 'final_language', 'popularity', 'year_exact', 'runtime_exact', 'vote_exact', 'arrondi_vote_exact', 'vote_count_mean'], axis = 1)
# df_LN_drop_NA = df_LN.dropna()
# X = df_LN_drop_NA
# y = pd.DataFrame(df_LN_drop_NA)

# def encodage_X(X, type='standard'):
#   X_num = X.select_dtypes('number')
#   X_cat = X.select_dtypes(['object', 'category', 'string'])
#   # Comme ça les dates, on n'y touche pas

#   if type == 'standard':
#     from sklearn.preprocessing import StandardScaler
#     SN = StandardScaler()
#     X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns)

#   else:
#     from sklearn.preprocessing import MinMaxScaler
#     SN = MinMaxScaler()
#     X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns)

#   X_cat_dummies = pd.get_dummies(X_cat)
#   X_encoded = pd.concat([X_num_SN.reset_index(), X_cat_dummies.reset_index()], axis=1)

#   return X_encoded, SN

# X_encoded, SN = encodage_X(X)

# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, train_size=0.8)

# from sklearn.linear_model import LinearRegression

# modele = LinearRegression()

# modele.fit(X_train, y_train)

# print(f"Score d'entrainement {modele.score(X_train, y_train)}")
# print(f"Score de test {modele.score(X_test, y_test)}")

# df_a_predire = df_test[df_test['vote_count_mean'] == 0].drop(['isOriginalTitle', 'ordering', 'film_id', 'id', 'revenue', 'budget', 'backdrop_path', 'homepage', 'overview', 'poster_path', 'status', 'tagline', 'video', 'titleType', 'types', 'attributes'], axis = 1)
# df_a_predire = df_a_predire.drap('vote_count_mean', axis = 1)
# df_quelonveutpredire = df_a_predire.dropna()

# def encodage_predict(df_a_predire):
#   X_num = df_a_predire.select_dtypes('number')
#   X_cat = df_a_predire.select_dtypes(['object', 'category', 'string'])

#   X_num_SN = pd.DataFrame(SN.transform(X_num), columns=X_num.columns).reset_index(drop=True)

#   X_cat_dummies = pd.get_dummies(X_cat).reset_index(drop=True)
#   X_encoded_predire = pd.concat([X_num_SN.reset_index(), X_cat_dummies.reset_index()], axis=1)

#   df_predict = X_encoded_predire

#   # DataFrame vide qui a les mêmes colonnes que X_encoded
#   df_final = pd.DataFrame(columns=X_encoded.columns)

#   # On veut que le DataFrame ait le même nombre de lignes que df_predict
#   df_final = df_final.reindex(index=df_predict.index)
#   # On met tous les NaN à False
#   df_final = df_final.fillna(False)

#   # On parcourt chaque colonne de df_predict
#   # Si la colonne est présente dans X_encoded alors on la garde
#   # Sinon, on la met à False
#   for column in df_predict.columns:
#     if column in X_encoded.columns:
#       df_final[column] = df_predict[column]

#   return df_final

# df_pour_prediction = encodage_predict(df_quelonveutpredire)

# df_quelonveutpredire['prediction'] = modele.predict(df_pour_prediction)
# df_quelonveutpredire
